In [ ]:
import pandas as pd
import os
import numpy as np
from scikit_posthocs import posthoc_dunn
from scipy import stats
# import the dataframe that you want to generate reports for
df = pd.read_csv(str(os. getcwd())+"/public_lib_tracker.csv")

In [ ]:
# test the normality 
column_values = df['number_of_total_trackers_homepage']
column_values = column_values.dropna()
# Perform Shapiro-Wilk test
stats.shapiro(column_values)

In [ ]:
# check if there is a difference among the categorical variables
df = pd.read_csv(str(os. getcwd())+"/public_lib_tracker.csv")
df.dropna(subset=['number_of_total_trackers_catalog', 'Current Automation System Name'], inplace=True)
group_freq = df['Current Automation System Name'].value_counts()
sorted_freq = group_freq.sort_values(ascending=False)
groups = sorted_freq.head(10).index.tolist()
results = stats.kruskal(*[df[df['Current Automation System Name'] == group]['number_of_total_trackers_catalog'] for group in groups])
results

In [ ]:
# perform dunn's test to compare between each groups 
filtered_df = df[df['Current Automation System Name'].isin(groups)]
dunn_results = posthoc_dunn(filtered_df, 
                            val_col = 'number_of_total_trackers_catalog',
                              group_col = 'Current Automation System Name',
                                p_adjust = 'holm')
dunn_results

In [ ]:
# get the ranking of the tracker rate
group_sums = filtered_df.groupby('Current Automation System Name')['number_of_total_trackers_catalog'].sum()

# Get the value counts of each group
group_counts = filtered_df['Current Automation System Name'].value_counts()

# Divide the value counts by the corresponding group sum
group_divided_counts = group_sums / group_counts

# Sort the divided counts in descending order
sorted_divided_counts = group_divided_counts.sort_values(ascending=False)

# Print the sorted divided counts
print(sorted_divided_counts)

In [ ]:
# get the frequency of the cookies' purpose
def parse_array_string(s):
    try:
        return eval(s) if isinstance(s, str) else None
    except (SyntaxError, NameError):
        return None

# Convert the column with array strings to actual arrays
df.iloc[:, 3:] = df.iloc[:, 3:].applymap(parse_array_string)

new_df = df.iloc[:, :3].copy()

# Apply a function to compute the length of arrays (if they exist) in each column
def compute_array_length(arr):
    if isinstance(arr, list):
        return len(arr)
    else:
        return None

# Apply the function to the remaining columns and assign the results to new columns
new_df = new_df.join(df.iloc[:, 3:].applymap(compute_array_length))

filtered_columns = new_df.filter(like='_homepage')

# Calculate the sum of each filtered column
column_sums = filtered_columns.sum()

# Calculate additional statistics for each filtered column
column_count = (filtered_columns > 0).sum()
column_median = filtered_columns.mask(filtered_columns == 0).median()
column_mean = filtered_columns.mask(filtered_columns == 0).mean()
column_std = filtered_columns.mask(filtered_columns == 0).std()

column_q1 = filtered_columns.mask(filtered_columns == 0).quantile(0.25)
column_q3 = filtered_columns.mask(filtered_columns == 0).quantile(0.75)

# Create a new DataFrame with the column names and statistics
sum_table = pd.DataFrame({
    'Column': column_sums.index,
    'Sum': column_sums.values,
    'Count': column_count.values,
    'Median': column_median.values,
    'Mean': column_mean.values,
    'Std': column_std.values,
    'Q1': column_q1.values,
    'Q3': column_q3.values
})

# Display the sum table
print(sum_table)